# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [37]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


# import statements
import re
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
#nltk.download(['punkt', 'wordnet', 'stopwords'])
#database
from sqlalchemy import create_engine

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.multioutput import MultiOutputClassifier

from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk.corpus import stopwords

import pickle

#constants
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'


In [2]:

# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_table("messages", con=engine)

X = df["message"]
Y = df[df.columns[4:]]

### 2. Normalization and  tokenization function to process your text data

In [3]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [41]:
classifier = MultiOutputClassifier(RandomForestClassifier(n_estimators=100))
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),# strings to token integer counts
    ('tfidf', TfidfTransformer()), # integer counts to weighted TF-IDF scores
    ('clf', classifier), # train on TF-IDF vectors w/ Naive Bayes classifier
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.3,random_state=101)


In [ ]:
pipeline.fit(X_train, y_train)


In [7]:
y_pred = pipeline.predict(X_test)


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
def get_report(y_test, y_pred):
    for i, col in enumerate(y_test):
        print(col)
        print(classification_report(y_test[col], y_pred[:, i]))

In [9]:
get_report(y_test, y_pred)

related
              precision    recall  f1-score   support

           0       0.74      0.35      0.48      1885
           1       0.82      0.96      0.88      5893
           2       0.67      0.25      0.36        56

    accuracy                           0.81      7834
   macro avg       0.74      0.52      0.57      7834
weighted avg       0.80      0.81      0.78      7834

request
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      6562
           1       0.89      0.43      0.58      1272

    accuracy                           0.90      7834
   macro avg       0.89      0.71      0.76      7834
weighted avg       0.90      0.90      0.88      7834

offer


/Applications/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7802
           1       0.00      0.00      0.00        32

    accuracy                           1.00      7834
   macro avg       0.50      0.50      0.50      7834
weighted avg       0.99      1.00      0.99      7834

aid_related
              precision    recall  f1-score   support

           0       0.72      0.93      0.81      4617
           1       0.82      0.47      0.60      3217

    accuracy                           0.74      7834
   macro avg       0.77      0.70      0.71      7834
weighted avg       0.76      0.74      0.72      7834

medical_help
              precision    recall  f1-score   support

           0       0.92      1.00      0.96      7216
           1       0.60      0.00      0.01       618

    accuracy                           0.92      7834
   macro avg       0.76      0.50      0.48      7834
weighted avg       0.90      0.92      0.88      

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      7510
           1       0.00      0.00      0.00       324

    accuracy                           0.96      7834
   macro avg       0.48      0.50      0.49      7834
weighted avg       0.92      0.96      0.94      7834

weather_related
              precision    recall  f1-score   support

           0       0.83      0.98      0.90      5679
           1       0.90      0.49      0.63      2155

    accuracy                           0.84      7834
   macro avg       0.87      0.73      0.77      7834
weighted avg       0.85      0.84      0.83      7834

floods
              precision    recall  f1-score   support

           0       0.93      1.00      0.97      7194
           1       0.94      0.20      0.33       640

    accuracy                           0.93      7834
   macro avg       0.94      0.60      0.65      7834
weighted avg       0.93      0.93      0.91      78

In [10]:
#print(confusion_matrix(y_test,predicted))
#print(classification_report(y_test,predicted))

### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x7fd418d340e0>)),
  ('tfidf', TfidfTransformer()),
  ('clf', RandomForestClassifier())],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x7fd418d340e0>),
 'tfidf': TfidfTransformer(),
 'clf': RandomForestClassifier(),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf__use_idf': True,
 'clf__bootstrap': True,
 'clf__ccp_alpha': 0.0,


In [26]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)


In [38]:
classifier = MultiOutputClassifier(RandomForestClassifier(n_estimators=100))

pipeline= Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor())
    ])),

    ('clf', classifier)
])

 # specify parameters for grid search
parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
        'features__text_pipeline__vect__max_features': (None, 5000, 10000),
        'features__text_pipeline__tfidf__use_idf': (True, False),
        'clf__n_estimators': [50, 100, 200],
        'clf__min_samples_split': [2, 3, 4],
        'features__transformer_weights': (
            {'text_pipeline': 1, 'starting_verb': 0.5},
            {'text_pipeline': 0.5, 'starting_verb': 1},
            {'text_pipeline': 0.8, 'starting_verb': 1},
        )
    }

cv = GridSearchCV(pipeline, param_grid=parameters)


In [40]:
cv.fit(X_train, y_train)


ValueError: Invalid parameter min_samples_split for estimator MultiOutputClassifier(estimator=RandomForestClassifier()). Check the list of available parameters with `estimator.get_params().keys()`.

In [ ]:
y_pred = cv.predict(X_test) 

In [ ]:
results = get_report(y_test, y_pred)
results

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
def get_report(y_test, y_pred):
    for i, col in enumerate(y_test):
        print(col)
        print(classification_report(y_test[col], y_pred[:, i]))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
#more here


### 9. Export your model as a pickle file

In [ ]:
# save the model to disk
filename = 'new_model.pkl'
pickle.dump(pipeline, open('models/' + filename, 'wb'))


In [ ]:
# test..

# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, Y_test)
print(result)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.